In [ ]:
from __future__ import absolute_import, division
from BashColors import C
import os
from os.path import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
contentPath=os.getcwd()
genPath=join(contentPath, 'DataGenerator')

from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

In [ ]:
# Load input data specific to an on-device ML app.
data = DataLoader.from_folder(genPath)
train_data, test_data = data.split(0.9)

In [ ]:
# Customize the TensorFlow model.
model = image_classifier.create(train_data, epochs=1)

In [ ]:
# Evaluate the model.
loss, accuracy = model.evaluate(test_data)

loss = round(loss, 4)
accuracy = round(accuracy, 1)
print(f'loss: {loss}\taccuracy: {accuracy * 100}%')

In [ ]:
# Export to Tensorflow Lite model and label file in `export_dir`.
# model.export(export_dir='/tmp/')
model.export(export_dir=contentPath)